In [ ]:
# Basic imports
import numpy as np
import pandas as pd
import seaborn as sns

from dataframe.csv_utils import (
    load_data_from_csv,
    get_filtered_data,
)

from constants import SORTED_BLOCK_NAMES, AUDIO_BLOCKS, COLOR_MAP, V_COLOR_MAP

In [ ]:
dir_name = "eeg_features2"
result = load_data_from_csv(dir_name)

In [ ]:
all_blocks = []
for b in SORTED_BLOCK_NAMES:
    all_blocks.extend([b] * 13)

result["condition"] = all_blocks * len(result.Subject.unique())
mask = result["condition"].isin(AUDIO_BLOCKS)
audio_only = result[mask]

In [ ]:
def get_category_counts(labels, default_thred: int, use_subject_mean: bool = False):
    threshold = default_thred if not use_subject_mean else np.mean(labels)
    low = 0
    for p in labels:
        if p < threshold:
            low += 1

    return len(labels) - low, low


label_dict = {
    "label_type": [],
    "subject": [],
    "global_h_count": [],
    "subject_mean_h_count": [],
    "global_l_count": [],
    "subject_mean_l_count": [],
}
for s in audio_only["Subject"].unique():
    for l in ['Valence', 'Arousal']:
        labels = audio_only[audio_only["Subject"] == s][l]
        default_thred = 0.6 if l == "Valence" else 0.5
        high_count_global, low_count_global = get_category_counts(labels, default_thred)
        high_count_subject, low_count_subject = get_category_counts(
            labels, default_thred, True
        )
        label_dict["subject"].append(s)
        label_dict["label_type"].append(l)
        label_dict["global_h_count"].append(high_count_global)
        label_dict["global_l_count"].append(low_count_global)
        label_dict["subject_mean_h_count"].append(high_count_subject)
        label_dict["subject_mean_l_count"].append(low_count_subject)    

In [ ]:
res = pd.DataFrame(label_dict)
subject_of_interest = [2001, 2003, 2017, 2026, 2028, 2033,  2037, 2041]
mask = (res["subject"].isin(subject_of_interest)) & (res['label_type'] == 'Valence')
res[mask]

In [ ]:
BREATH_BLOCKS = [
    "breath_hvha",
    "breath_hvla",
    "breath_nvha",
    "breath_nvla",
]
mask = result["condition"].isin(BREATH_BLOCKS)
breath_only = result[mask]

REST_BLOCKS = ["meditation", "mind wandering",]
mask = result["condition"].isin(REST_BLOCKS)
rest_only = result[mask]

In [ ]:
sns.scatterplot(data=audio_only, x="Valence", y="Arousal", hue="condition")

In [ ]:
g = sns.relplot(
    data=audio_only,
    x="Valence",
    y="Arousal",
    col="Subject",
    hue="condition",
    col_wrap=5,
    palette=COLOR_MAP,
    kind="scatter",
)

sns.move_legend(g, "upper right")

for _, ax in g.axes_dict.items():
    ax.axhline(y = 0.5, color = "grey", linestyle = "dashed")
    ax.axvline(x = 0.5, color = "grey", linestyle = "dashed") # To the right